In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
!ls

MANIFEST.in                           gceline_frontend_streamlit.ipynb
Makefile                              notebooks
README.md                             raw_data
WorkforceSentimentMonitoring          requirements.txt
WorkforceSentimentMonitoring.egg-info scripts
app.py                                setup.py
build                                 test_data
dist                                  tests


In [3]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [5]:
from WorkforceSentimentMonitoring.data import get_data, merge, holdout, encode_target, \
drop_wrong_language, get_prepaired_data

In [9]:
from WorkforceSentimentMonitoring.feature_engineering import SCORE_COLS, FEATURE_COLS

In [90]:
from WorkforceSentimentMonitoring.encoders import Preprocessor, FeatureEngineer, CustomMinMaxScaler, PredictionFeaturesExtractor

# Load Data

In [13]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Identifying entries in other languages...
Drop 439 entries? [y] / n
y
Dropping 439 entries...
Process completed.
Splitting train and test...
Encoding targets...
Done!


In [100]:
display(X_train.head())
display(y_train.head())

,summary,positives,negatives,advice_to_mgmt,review
0,Pathways Operations Manager,A very fast paced and customer focused company,Not a very employee friendly company,NaN,Pathways Operations Manager A very fast paced ...
1,Ok for short term work,Benefits from 1st day of employment,"Physically demanding, long shifts, high produc...",NaN,Ok for short term work Benefits from 1st day o...
2,Great People,Work with people who are passionate about what...,"Need open offices, many offices don't have any...",Encourage people to move between teams. Breadt...,Great People Work with people who are passiona...
3,Constantly making history,Great work culture Very Direct and no politick...,Constant change of plans Work life Balance,NaN,Constantly making history Great work culture V...
4,What you would expect,Pay is good for job being done,"retail hours, getting time off",NaN,What you would expect Pay is good for job bein...


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,0,2,2,2,2,2
1,0,1,1,2,1,1
2,2,2,2,2,2,2
3,1,2,2,2,2,2
4,1,2,1,2,2,2


In [101]:
X_train.isnull().sum()

summary              62
positives             0
negatives             1
advice_to_mgmt    15691
review                0
dtype: int64

In [102]:
X_test.isnull().sum()

summary             18
positives            0
negatives            0
advice_to_mgmt    6748
review               0
dtype: int64

In [17]:
import pandas as pd

In [18]:
X = pd.concat([X_train, X_test])

In [19]:
y = pd.concat([y_train, y_test])

In [20]:
display(X.head())
display(y.head())

,summary,positives,negatives,advice_to_mgmt,review
0,Pathways Operations Manager,A very fast paced and customer focused company,Not a very employee friendly company,NaN,Pathways Operations Manager A very fast paced ...
1,Ok for short term work,Benefits from 1st day of employment,"Physically demanding, long shifts, high produc...",NaN,Ok for short term work Benefits from 1st day o...
2,Great People,Work with people who are passionate about what...,"Need open offices, many offices don't have any...",Encourage people to move between teams. Breadt...,Great People Work with people who are passiona...
3,Constantly making history,Great work culture Very Direct and no politick...,Constant change of plans Work life Balance,NaN,Constantly making history Great work culture V...
4,What you would expect,Pay is good for job being done,"retail hours, getting time off",NaN,What you would expect Pay is good for job bein...


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,0,2,2,2,2,2
1,0,1,1,2,1,1
2,2,2,2,2,2,2
3,1,2,2,2,2,2
4,1,2,1,2,2,2


In [66]:
df = pd.concat([X, y], axis=1)

In [67]:
df.dropna(inplace=True)

In [93]:
df_copy = df.copy()

In [98]:
df_copy.to_csv('raw_data/pos_neg_df.csv', encoding='utf-8', index=False)

## Group in 2 classes

In [129]:
# remap scores into 2 class targets (leave score 3 as '3')
'''def remap(data, dict_labels):
    for key, values in dict_labels.items():
        data.replace({key: {1: '0', 2: '0', 3: '1', 4: '2', 5: '2'}}, inplace=True)
    return data'''

In [114]:
#small_target.apply(lambda row: remap(small_target, small_target), axis=1)

I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping c

I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping 

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

...

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapp

I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-valu

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

0            work-balance culture-values career-opport...
1            work-balance culture-values career-opport...
2            work-balance culture-values career-opport...
3            work-balance culture-values career-opport...
4            work-balance culture-values career-opport...
                              ...                        
9995         work-balance culture-values career-opport...
9996         work-balance culture-values career-opport...
9997         work-balance culture-values career-opport...
9998         work-balance culture-values career-opport...
9999         work-balance culture-values career-opport...
Length: 10000, dtype: object

In [125]:
#small_target = small_target.astype('uint8')

In [ ]:
#small_target

# Prepare 2-class target

In [34]:
# transform scores into 2 classes: 0 => negative; 1 => positive
#target.apply(lambda row: remap(target, target), axis=1)

In [174]:
X_train_copy = X_train.copy()
y_train_copy = y_train.copy()
X_test_copy = X_test.copy()
y_test_copy = y_test.copy()

In [175]:
X_train_copy

,summary,positives,negatives,advice_to_mgmt,review
0,Pathways Operations Manager,A very fast paced and customer focused company,Not a very employee friendly company,NaN,Pathways Operations Manager A very fast paced ...
1,Ok for short term work,Benefits from 1st day of employment,"Physically demanding, long shifts, high produc...",NaN,Ok for short term work Benefits from 1st day o...
2,Great People,Work with people who are passionate about what...,"Need open offices, many offices don't have any...",Encourage people to move between teams. Breadt...,Great People Work with people who are passiona...
3,Constantly making history,Great work culture Very Direct and no politick...,Constant change of plans Work life Balance,NaN,Constantly making history Great work culture V...
4,What you would expect,Pay is good for job being done,"retail hours, getting time off",NaN,What you would expect Pay is good for job bein...
...,...,...,...,...,...
36658,The place is the what it cracked up to be,steady hours and you get to work with a lot of...,The Pay suck ball's big time for amount effort...,Amazon makes way to much money to be dashing 1...,The place is the what it cracked up to be stea...
36659,buyer risk investigator,- good working environment - structured organi...,-different shift timings -rotational shifts ev...,when employees work in unusual timings they sh...,buyer risk investigator - good working environ...
36660,Better hope you end up in a better team than I...,"Great benefits and pay, especially coming for ...",5 managers in the first year and a half. No di...,Don't leave your entry level hires high and dr...,Better hope you end up in a better team than I...
36661,Caught in the Gears,You will learn. A lot. This company has some o...,Your employment will be a blessing or a condem...,"Stop the churn. Yes, you need to raise the bar...",Caught in the Gears You will learn. A lot. Thi...


In [176]:
y_train_copy

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,0,2,2,2,2,2
1,0,1,1,2,1,1
2,2,2,2,2,2,2
3,1,2,2,2,2,2
4,1,2,1,2,2,2
...,...,...,...,...,...,...
36658,0,1,0,0,0,0
36659,1,2,2,1,2,1
36660,2,0,0,2,0,2
36661,0,2,1,0,0,0


## Prepare Features

In [177]:
from WorkforceSentimentMonitoring.feature_engineering import get_lengths,\
get_linear_regression_cols, get_subjectivity_polarity_columns,\
getPolarity, getSubjectivity, add_multinomial_nb_prediction_feature

In [179]:
preprocessor = Preprocessor()

In [180]:
X_train_copy = preprocessor.fit_transform(X_train_copy)

In [181]:
X_test_copy = preprocessor.fit_transform(X_test_copy)

In [182]:
feature_engineering = FeatureEngineer()

In [183]:
X_train_copy = feature_engineering.fit_transform(X_train_copy)

100%|██████████| 5/5 [01:20<00:00, 16.00s/it]


In [184]:
X_train_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,pathway operation manager,fast paced customer focused company,employee friendly company,NaN,pathway operation manager fast paced customer ...,25,35,25,0,91,0.000000,0.00,0.600000,0.200000,0.500000,0.375000,NaN,NaN,0.550000,0.287500
1,ok short term work,benefit st day employment,physically demanding long shift high productiv...,NaN,ok short term work benefit st day employment p...,18,25,55,0,104,0.400000,0.25,0.000000,0.000000,0.360952,0.036667,NaN,NaN,0.376571,0.122000
2,great people,work people passionate care greater good fanta...,need open office many office sunlight,encourage people move team breadth knowledge b...,great people work people passionate care great...,12,120,37,125,297,0.750000,0.80,0.605357,0.376786,0.500000,0.250000,0.452273,0.093182,0.556899,0.307873
3,constantly making history,great work culture direct politicking customer...,constant change plan work life balance,NaN,constantly making history great work culture d...,25,55,38,0,124,0.333333,0.00,0.716667,0.133333,0.333333,0.000000,NaN,NaN,0.563333,0.080000
4,would expect,pay good job done,retail hour getting time,NaN,would expect pay good job done retail hour get...,12,17,24,0,59,0.000000,0.00,0.600000,0.700000,0.000000,0.000000,NaN,NaN,0.600000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36658,place cracked,steady hour get work lot interesting funny people,pay suck ball big time amount effort expected ...,amazon make way much money dashing buck hour a...,place cracked steady hour get work lot interes...,13,49,146,100,311,0.000000,0.00,0.666667,0.305556,0.425000,-0.075000,0.180952,0.033333,0.424286,0.071667
36659,buyer risk investigator,good working environment structured organizati...,different shift timing rotational shift every ...,employee work unusual timing provided proper f...,buyer risk investigator good working environme...,23,113,106,142,387,0.000000,0.00,0.533333,0.466667,0.600000,0.000000,0.466667,0.066667,0.514286,0.228571
36660,better hope end better team,great benefit pay especially coming coming str...,manager first year half direction given someon...,leave entry level hire high dry sure thing cha...,better hope end better team great benefit pay ...,27,126,205,98,459,0.500000,0.50,0.530000,0.440000,0.433333,0.080556,0.669270,0.261524,0.533131,0.277275
36661,caught gear,learn lot company challenging exciting technol...,employment blessing condemnation depending alm...,stop churn yes need raise bar time actually ca...,caught gear learn lot company challenging exci...,11,219,506,431,1170,0.000000,0.00,0.750000,0.466667,0.472443,-0.035038,0.530127,0.006506,0.561896,0.096455


In [185]:
X_test_copy = feature_engineering.fit_transform(X_test_copy)

100%|██████████| 5/5 [00:36<00:00,  7.26s/it]


In [186]:
X_train_copy.dropna(inplace=True)

In [187]:
X_train_copy.reset_index(drop=True, inplace=True)

In [188]:
X_test_copy.dropna(inplace=True)

In [189]:
X_test_copy.reset_index(drop=True, inplace=True)

In [190]:
X_train_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,great people,work people passionate care greater good fanta...,need open office many office sunlight,encourage people move team breadth knowledge b...,great people work people passionate care great...,12,120,37,125,297,0.750000,0.8,0.605357,0.376786,0.500000,0.250000,0.452273,0.093182,0.556899,0.307873
1,biggest gap value real life behavior,lot knowledge flexible hour office process rea...,key problem company lost true leader existing ...,make real go home,biggest gap value real life behavior lot knowl...,36,54,646,17,756,0.300000,0.2,0.600000,0.700000,0.511111,0.071032,0.300000,0.200000,0.497222,0.107986
2,amazing amazon,immense carrier opportunity growth opportunity...,working fc first difficult week get habitual s...,work career development define growth path,amazing amazon immense carrier opportunity gro...,14,314,218,42,591,0.900000,0.6,0.612500,0.400000,0.331313,0.014394,0.000000,0.000000,0.519192,0.259091
3,make sure know role applying,employee food benefit perk shuttle basically e...,company become way vertical performance evalua...,stop putting people bell curve recognize stell...,make sure know role applying employee food ben...,28,65,121,95,312,0.888889,0.5,0.000000,0.000000,0.600000,-0.100000,0.250000,0.250000,0.579630,0.216667
4,ehhh,good pay somewhat nice people,hired ’ always something,’ rude employee maybe ’ want actually stay work,ehhh good pay somewhat nice people hired ’ alw...,4,29,24,47,107,0.000000,0.0,0.800000,0.650000,0.000000,0.000000,0.350000,-0.150000,0.575000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20930,sale manager,great company leadership great benefit smart p...,overrun process policy lack empowerment,remove unnecessary process empower seller asso...,sale manager great company leadership great be...,12,51,39,51,156,0.000000,0.0,0.714286,0.604762,0.000000,0.000000,0.900000,-0.400000,0.760714,0.353571
20931,place cracked,steady hour get work lot interesting funny people,pay suck ball big time amount effort expected ...,amazon make way much money dashing buck hour a...,place cracked steady hour get work lot interes...,13,49,146,100,311,0.000000,0.0,0.666667,0.305556,0.425000,-0.075000,0.180952,0.033333,0.424286,0.071667
20932,buyer risk investigator,good working environment structured organizati...,different shift timing rotational shift every ...,employee work unusual timing provided proper f...,buyer risk investigator good working environme...,23,113,106,142,387,0.000000,0.0,0.533333,0.466667,0.600000,0.000000,0.466667,0.066667,0.514286,0.228571
20933,better hope end better team,great benefit pay especially coming coming str...,manager first year half direction given someon...,leave entry level hire high dry sure thing cha...,better hope end better team great benefit pay ...,27,126,205,98,459,0.500000,0.5,0.530000,0.440000,0.433333,0.080556,0.669270,0.261524,0.533131,0.277275


In [191]:
X_test_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,consultant product,good benefit great company environment,sometimes feel like number,could organisation,consultant product good benefit great company ...,18,38,26,18,103,0.000000,0.00,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000
1,excellent benefit frustrating sometimes,great place term benefit freedom work,bit cumbersome due size,start listening engineer,excellent benefit frustrating sometimes great ...,39,37,23,24,126,0.950000,0.30,0.750000,0.800000,0.375000,-0.125000,0.000000,0.000000,0.756250,0.318750
2,work le number people,people really smart,work life balance pager duty seattle weather,much customer obsession time obsessed employee,work le number people people really smart work...,21,19,44,46,133,0.000000,0.00,0.642857,0.214286,0.000000,0.000000,0.600000,-0.150000,0.614286,-0.028571
3,knowledgeable coworkers incompetent management,working latest technology amazing training wor...,management high turn always insecure lack tech...,promotion within would keep unified vision sto...,knowledgeable coworkers incompetent management...,46,105,120,64,338,0.366667,-0.35,0.900000,0.600000,0.707500,-0.170000,0.000000,0.000000,0.746944,0.185000
4,software development engineer sde ii,good salary broad range technology challenging...,extra compared large software internet company...,well documented amazonians leave within two ye...,software development engineer sde ii good sala...,36,430,311,298,1078,0.000000,0.00,0.712686,0.253757,0.354014,0.188844,0.500000,0.500000,0.599211,0.245084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,poor ethic,reasonable salary brand value international pr...,poor management transparency top speech limite...,wakeup know happening ground show empathy care...,poor ethic reasonable salary brand value inter...,10,83,137,83,316,0.600000,-0.40,0.425000,0.225000,0.335714,0.032143,0.000000,0.000000,0.404762,0.069841
8955,big slow,excellent pay smart people great benefit gadge...,big slow many silo distributed hard tell going...,fire half middle management ditch perf system ...,big slow excellent pay smart people great bene...,8,95,121,102,329,0.250000,-0.15,0.778571,0.722857,0.290278,-0.031944,0.083333,-0.083333,0.420079,0.197063
8956,sale,flexible hour good work life balance,weak senior management la office manager train...,promote assign someone people management role ...,sale flexible hour good work life balance weak...,4,36,115,132,290,0.000000,0.00,0.600000,0.700000,0.812500,-0.187500,0.850000,-0.050000,0.768750,0.068750
8957,hardware intern,free dinner great pay nice campus apple produc...,encourage stay past limited term impact create,many level hierarchy expected large company ne...,hardware intern free dinner great pay nice cam...,15,56,46,85,205,0.000000,0.00,0.850000,0.600000,0.196429,-0.160714,0.442857,0.204762,0.533929,0.261607


In [192]:
df_train = pd.concat([X_train_copy, y_train_copy], axis=1)

In [193]:
df_test = pd.concat([X_test_copy, y_test_copy], axis=1)

In [194]:
df_train.dropna(inplace=True)

In [206]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from tqdm import tqdm
from sklearn.naive_bayes import MultinomialNB

In [209]:
scores_dict= {}
for score in y_test_copy.columns:
    result_scores = {}
    for feature in df_train.select_dtypes('object').columns:
        # instantiate vectorizer
        vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,7))
        # vectorize train set
        X_train = vectorizer.fit_transform(df_train[feature].astype('U'))
        # vectorize test set
        X_test = vectorizer.transform(df_test[feature].astype('U'))
        # select target
        target = y_test_copy[score]
        # instantiate model
        model = MultinomialNB()
        # fit model
        model.fit(X_train, y_train_copy[score])
        # predict with train and test sets and store predictions in new col
        df_train[f"{feature}_{score}_nb"] = model.predict(X_train)
        df_test[f"{feature}_{score}_nb"] = model.predict(X_test)
        # evaluate model and append results in dictionary
        result_scores[f'{feature}'] = model.score(X_test, target)
    # append results to scores dictionary
    scores_dict[f'{score}'] = result_scores
return df_train, df_test, scores_dict

ValueError: Found input variables with inconsistent numbers of samples: [20935, 36663]

In [197]:
df_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,great people,work people passionate care greater good fanta...,need open office many office sunlight,encourage people move team breadth knowledge b...,great people work people passionate care great...,12.0,120.0,37.0,125.0,297.0,...,0.452273,0.093182,0.556899,0.307873,0,2,2,2,2,2
1,biggest gap value real life behavior,lot knowledge flexible hour office process rea...,key problem company lost true leader existing ...,make real go home,biggest gap value real life behavior lot knowl...,36.0,54.0,646.0,17.0,756.0,...,0.300000,0.200000,0.497222,0.107986,0,1,1,2,1,1
2,amazing amazon,immense carrier opportunity growth opportunity...,working fc first difficult week get habitual s...,work career development define growth path,amazing amazon immense carrier opportunity gro...,14.0,314.0,218.0,42.0,591.0,...,0.000000,0.000000,0.519192,0.259091,2,2,2,2,2,2
3,make sure know role applying,employee food benefit perk shuttle basically e...,company become way vertical performance evalua...,stop putting people bell curve recognize stell...,make sure know role applying employee food ben...,28.0,65.0,121.0,95.0,312.0,...,0.250000,0.250000,0.579630,0.216667,1,2,2,2,2,2
4,ehhh,good pay somewhat nice people,hired ’ always something,’ rude employee maybe ’ want actually stay work,ehhh good pay somewhat nice people hired ’ alw...,4.0,29.0,24.0,47.0,107.0,...,0.350000,-0.150000,0.575000,0.250000,1,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20930,sale manager,great company leadership great benefit smart p...,overrun process policy lack empowerment,remove unnecessary process empower seller asso...,sale manager great company leadership great be...,12.0,51.0,39.0,51.0,156.0,...,0.900000,-0.400000,0.760714,0.353571,1,1,2,2,2,2
20931,place cracked,steady hour get work lot interesting funny people,pay suck ball big time amount effort expected ...,amazon make way much money dashing buck hour a...,place cracked steady hour get work lot interes...,13.0,49.0,146.0,100.0,311.0,...,0.180952,0.033333,0.424286,0.071667,2,2,2,2,2,1
20932,buyer risk investigator,good working environment structured organizati...,different shift timing rotational shift every ...,employee work unusual timing provided proper f...,buyer risk investigator good working environme...,23.0,113.0,106.0,142.0,387.0,...,0.466667,0.066667,0.514286,0.228571,0,0,1,1,1,1
20933,better hope end better team,great benefit pay especially coming coming str...,manager first year half direction given someon...,leave entry level hire high dry sure thing cha...,better hope end better team great benefit pay ...,27.0,126.0,205.0,98.0,459.0,...,0.669270,0.261524,0.533131,0.277275,0,2,2,2,2,2


In [198]:
df_test

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,consultant product,good benefit great company environment,sometimes feel like number,could organisation,consultant product good benefit great company ...,18.0,38.0,26.0,18.0,103.0,...,0.000000,0.000000,0.675000,0.750000,2,2,0,2,2,2
1,excellent benefit frustrating sometimes,great place term benefit freedom work,bit cumbersome due size,start listening engineer,excellent benefit frustrating sometimes great ...,39.0,37.0,23.0,24.0,126.0,...,0.000000,0.000000,0.756250,0.318750,2,2,2,1,2,2
2,work le number people,people really smart,work life balance pager duty seattle weather,much customer obsession time obsessed employee,work le number people people really smart work...,21.0,19.0,44.0,46.0,133.0,...,0.600000,-0.150000,0.614286,-0.028571,0,0,2,2,0,0
3,knowledgeable coworkers incompetent management,working latest technology amazing training wor...,management high turn always insecure lack tech...,promotion within would keep unified vision sto...,knowledgeable coworkers incompetent management...,46.0,105.0,120.0,64.0,338.0,...,0.000000,0.000000,0.746944,0.185000,1,2,2,2,2,2
4,software development engineer sde ii,good salary broad range technology challenging...,extra compared large software internet company...,well documented amazonians leave within two ye...,software development engineer sde ii good sala...,36.0,430.0,311.0,298.0,1078.0,...,0.500000,0.500000,0.599211,0.245084,2,2,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,poor ethic,reasonable salary brand value international pr...,poor management transparency top speech limite...,wakeup know happening ground show empathy care...,poor ethic reasonable salary brand value inter...,10.0,83.0,137.0,83.0,316.0,...,0.000000,0.000000,0.404762,0.069841,2,2,2,1,2,2
8955,big slow,excellent pay smart people great benefit gadge...,big slow many silo distributed hard tell going...,fire half middle management ditch perf system ...,big slow excellent pay smart people great bene...,8.0,95.0,121.0,102.0,329.0,...,0.083333,-0.083333,0.420079,0.197063,2,2,2,2,2,2
8956,sale,flexible hour good work life balance,weak senior management la office manager train...,promote assign someone people management role ...,sale flexible hour good work life balance weak...,4.0,36.0,115.0,132.0,290.0,...,0.850000,-0.050000,0.768750,0.068750,0,1,1,2,0,1
8957,hardware intern,free dinner great pay nice campus apple produc...,encourage stay past limited term impact create,many level hierarchy expected large company ne...,hardware intern free dinner great pay nice cam...,15.0,56.0,46.0,85.0,205.0,...,0.442857,0.204762,0.533929,0.261607,2,2,2,2,0,0


In [195]:
df_test.dropna(inplace=True)

In [199]:
df_train, df_test, scores_dict = add_multinomial_nb_prediction_feature(df_train, df_test, y_train_copy, y_test_copy)

ValueError: Found input variables with inconsistent numbers of samples: [20935, 36663]

In [170]:
df_train

,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,12.0,120.0,37.0,125.0,297.0,0.750000,0.8,0.605357,0.376786,0.500000,...,0.452273,0.093182,0.556899,0.307873,0,2,2,2,2,2
1,36.0,54.0,646.0,17.0,756.0,0.300000,0.2,0.600000,0.700000,0.511111,...,0.300000,0.200000,0.497222,0.107986,0,1,1,2,1,1
2,14.0,314.0,218.0,42.0,591.0,0.900000,0.6,0.612500,0.400000,0.331313,...,0.000000,0.000000,0.519192,0.259091,2,2,2,2,2,2
3,28.0,65.0,121.0,95.0,312.0,0.888889,0.5,0.000000,0.000000,0.600000,...,0.250000,0.250000,0.579630,0.216667,1,2,2,2,2,2
4,4.0,29.0,24.0,47.0,107.0,0.000000,0.0,0.800000,0.650000,0.000000,...,0.350000,-0.150000,0.575000,0.250000,1,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20930,12.0,51.0,39.0,51.0,156.0,0.000000,0.0,0.714286,0.604762,0.000000,...,0.900000,-0.400000,0.760714,0.353571,1,1,2,2,2,2
20931,13.0,49.0,146.0,100.0,311.0,0.000000,0.0,0.666667,0.305556,0.425000,...,0.180952,0.033333,0.424286,0.071667,2,2,2,2,2,1
20932,23.0,113.0,106.0,142.0,387.0,0.000000,0.0,0.533333,0.466667,0.600000,...,0.466667,0.066667,0.514286,0.228571,0,0,1,1,1,1
20933,27.0,126.0,205.0,98.0,459.0,0.500000,0.5,0.530000,0.440000,0.433333,...,0.669270,0.261524,0.533131,0.277275,0,2,2,2,2,2


In [171]:
df_test

,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,18.0,38.0,26.0,18.0,103.0,0.000000,0.00,0.675000,0.750000,0.000000,...,0.000000,0.000000,0.675000,0.750000,2,2,0,2,2,2
1,39.0,37.0,23.0,24.0,126.0,0.950000,0.30,0.750000,0.800000,0.375000,...,0.000000,0.000000,0.756250,0.318750,2,2,2,1,2,2
2,21.0,19.0,44.0,46.0,133.0,0.000000,0.00,0.642857,0.214286,0.000000,...,0.600000,-0.150000,0.614286,-0.028571,0,0,2,2,0,0
3,46.0,105.0,120.0,64.0,338.0,0.366667,-0.35,0.900000,0.600000,0.707500,...,0.000000,0.000000,0.746944,0.185000,1,2,2,2,2,2
4,36.0,430.0,311.0,298.0,1078.0,0.000000,0.00,0.712686,0.253757,0.354014,...,0.500000,0.500000,0.599211,0.245084,2,2,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,10.0,83.0,137.0,83.0,316.0,0.600000,-0.40,0.425000,0.225000,0.335714,...,0.000000,0.000000,0.404762,0.069841,2,2,2,1,2,2
8955,8.0,95.0,121.0,102.0,329.0,0.250000,-0.15,0.778571,0.722857,0.290278,...,0.083333,-0.083333,0.420079,0.197063,2,2,2,2,2,2
8956,4.0,36.0,115.0,132.0,290.0,0.000000,0.00,0.600000,0.700000,0.812500,...,0.850000,-0.050000,0.768750,0.068750,0,1,1,2,0,1
8957,15.0,56.0,46.0,85.0,205.0,0.000000,0.00,0.850000,0.600000,0.196429,...,0.442857,0.204762,0.533929,0.261607,2,2,2,2,0,0


In [172]:
scores_dict

{'work-balance': {},
 'culture-values': {},
 'career-opportunities': {},
 'comp-benefits': {},
 'senior-mgmt': {},
 'overall': {}}

In [150]:
scaler = CustomMinMaxScaler()

In [72]:
X_train_copy_scaled = scaler.fit_transform(X_train_copy)

In [165]:
X_train_copy

,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,12,120,37,125,297,0.750000,0.8,0.605357,0.376786,0.500000,0.250000,0.452273,0.093182,0.556899,0.307873
1,36,54,646,17,756,0.300000,0.2,0.600000,0.700000,0.511111,0.071032,0.300000,0.200000,0.497222,0.107986
2,14,314,218,42,591,0.900000,0.6,0.612500,0.400000,0.331313,0.014394,0.000000,0.000000,0.519192,0.259091
3,28,65,121,95,312,0.888889,0.5,0.000000,0.000000,0.600000,-0.100000,0.250000,0.250000,0.579630,0.216667
4,4,29,24,47,107,0.000000,0.0,0.800000,0.650000,0.000000,0.000000,0.350000,-0.150000,0.575000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20930,12,51,39,51,156,0.000000,0.0,0.714286,0.604762,0.000000,0.000000,0.900000,-0.400000,0.760714,0.353571
20931,13,49,146,100,311,0.000000,0.0,0.666667,0.305556,0.425000,-0.075000,0.180952,0.033333,0.424286,0.071667
20932,23,113,106,142,387,0.000000,0.0,0.533333,0.466667,0.600000,0.000000,0.466667,0.066667,0.514286,0.228571
20933,27,126,205,98,459,0.500000,0.5,0.530000,0.440000,0.433333,0.080556,0.669270,0.261524,0.533131,0.277275


In [77]:
X_train_copy_final = extractor.fit_transform(X_train_copy_scaled, y_train_copy)

AttributeError: 'NoneType' object has no attribute 'transform'

# Train model

In [80]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
def train_RanForest(X_train, y_train, X_test, y_test, SCORE_COLS):
    prediction_scores_dict = {}
    for target in tqdm(SCORE_COLS):
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train[target])
        prediction_scores[target] = model.score(X_test, y_test[target])

    return model, prediction_scores_dict